In [10]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
# Load environment variables from .env
load_dotenv()



True

In [11]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("what is the main purpose of whatfix company ")
print(response.content)

Whatfix is a digital adoption platform (DAP) company that provides a suite of tools to help organizations improve the onboarding, training, and support of their employees, customers, or users on various software applications, websites, and digital products.

The main purpose of Whatfix is to:

1. **Simplify complex software applications**: Whatfix helps to break down complex software applications into simple, easy-to-follow instructions, making it easier for users to learn and use them.
2. **Improve user adoption and engagement**: By providing contextual, in-app guidance and support, Whatfix enables users to quickly get up to speed with new software applications, reducing the time it takes to become proficient.
3. **Reduce support queries and tickets**: Whatfix's interactive guides and tutorials help users to resolve common issues and answer frequently asked questions, reducing the number of support queries and tickets.
4. **Enhance overall user experience**: By providing personalized,

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://medusajs.com/careers/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Medusa - Join UsProductSolutionsDevelopersPricingWe're hiringDocsSign up MenuDocs·Sign upCareersChange how
commerce is builtBuilding a generational company requires exceptional, hard-working people. We are tackling the complexities of commerce infrastructure that no one else has dared to.Join the teamSebastian RindomOliver JuhlNicklas Gellner+15Engineering·Europe & US East CoastSoftware Engineer, Open-sourceAs apart of our open-source team, you will help improve our commerce platform and framework tooling to benefit Medusa developers and merchants.Operations·Europe & US East CoastFounders Associate; GTM, Sales & OpsStep into the engine room of a fast-growing tech startup and help shape Medusa’s commercial future while learning how to market and scale a category defining commerce platform.Engineering·Europe & US East CoastTechnical Support EngineerAs a Technical Support Engineer, you will play a critical role in helping our users build world-class commerce experiencesEngineering·Europe 

In [13]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """ 
    ### SCRAPED TEXT FROM WEBISTE
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website git.
    Your job is to extract the job postings and return then in JSON format containing
    following keys: `role`,`experience`,`skills` and `description. 
    Only return the Valid JSON.
    ### VALID JSON (NO PREAMBLE):

    """
    
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer, Open-source',
  'experience': '',
  'skills': '',
  'description': 'As apart of our open-source team, you will help improve our commerce platform and framework tooling to benefit Medusa developers and merchants.'},
 {'role': 'Founders Associate; GTM, Sales & Ops',
  'experience': '',
  'skills': '',
  'description': 'Step into the engine room of a fast-growing tech startup and help shape Medusa’s commercial future while learning how to market and scale a category defining commerce platform.'},
 {'role': 'Technical Support Engineer',
  'experience': '',
  'skills': '',
  'description': 'As a Technical Support Engineer, you will play a critical role in helping our users build world-class commerce experiences'},
 {'role': 'Solutions Engineer',
  'experience': '',
  'skills': '',
  'description': 'As a Solutions Engineer, you will help Medusa users succeed and bring back learnings to the core team.'}]

In [15]:
type(json_res)

list

In [16]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [21]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [20]:
job = json_res[0]  
print(job['skills']) 

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
       ### INSTRUCTION:
       You are Mohan, a Business Development Executive at Whatfix. Whatfix is a leading Digital Adoption Platform that empowers organizations to simplify user onboarding, training, and software adoption through in-app guidance and automation.  
       Over the years, Whatfix has helped global enterprises enhance user productivity, reduce support costs, and accelerate digital transformation.  
       Your job is to write a personalized cold email to the client regarding the job mentioned above, highlighting how Whatfix can address their onboarding and training challenges for this role.  
       Also add the most relevant ones from the following links to showcase Whatfix's portfolio and success stories: {link_list}  
       Remember, you are Mohan, BDE at Whatfix.  
       Do not provide a preamble.  
       ### EMAIL (NO PREAMBLE):

        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Simplify Onboarding and Training for Your Open-Source Software Engineers

Dear Hiring Manager,

I came across the job description for a Software Engineer, Open-source, and I was impressed by the role's focus on improving the commerce platform and framework tooling for Medusa developers and merchants. As a Business Development Executive at Whatfix, I believe our Digital Adoption Platform can help address the onboarding and training challenges that come with this position.

Our platform empowers organizations to simplify user onboarding, training, and software adoption through in-app guidance and automation. With Whatfix, you can create personalized learning paths, automate repetitive tasks, and provide real-time support to your software engineers, ensuring they become productive quickly.

I'd like to highlight a few success stories from our portfolio that demonstrate the impact of our platform. For instance, our work with Android TV ([https://example.com/android-tv-portfolio](h